In [20]:
import os, sys, email
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import bs4 as bs

import os
import itertools
from collections import Counter
from wordcloud import WordCloud
import re
import string

from tqdm import tqdm # for examining loops
%matplotlib inline
import seaborn as sns; sns.set_style('whitegrid')
import io
import pandas as pd


import nltk 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer 

from transformers import pipeline
import torch
torch.cuda.is_available()
from transformers import AutoTokenizer


nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\adity\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adity\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\adity\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [21]:
journal_df = pd.read_csv('data/journals.csv')
journal_df.head()

,authors,title,abstract,journal_name
0,Yu-Xiao Wang; Yue Xin; Jun-Yi Yin; Xiao-Jun Hu...,Revealing the architecture and solution proper...,Macrolepiota albuminosa (Berk.) Pegler is abun...,Food Chemistry
1,Yu-Xiao Wang; Yue Xin; Xiao-Jun Huang; Jun-Yi ...,A branched galactoglucan with flexible chains ...,A homogeneous galactoglucan was purified from ...,Food Chemistry
2,Yu-Xiao Wang; Ting Zhang; Jun-Yi Yin; Xiao-Jun...,Structural characterization and rheological pr...,A homogeneous beta-glucan (JHMP-70) was obtain...,Food Hydrocolloids
3,Yu-Xin Gu; Tian-Ci Yan; Zi-Xuan Yue; Min-Hui L...,Dispersive micro-solid-phase extraction of aca...,A novel dispersive micro-solid-phase extractio...,Food Analytical Methods
4,Yu-Xue Xu; Ze-Dong Jiang; Xi-Ping Du; Ming-Jin...,The identification of biotransformation pathwa...,The yeast Saccharomyces cerevisiae is effectiv...,Food Chemistry


In [22]:
# create a dataframe of percentage of null values
null_dict = (dict(journal_df.isna().mean().round(4)*100))
null_df = pd.DataFrame.from_dict(null_dict, orient="index").reset_index()
null_df.columns = ['col', 'percentage']
null_df = null_df.sort_values('percentage',ascending=True)
null_df.head(20)

,col,percentage
0,authors,0.00
1,title,0.00
3,journal_name,0.00
2,abstract,0.12


In [26]:
# Drop rows where the "abstract" column contains NaN values
journal_df = journal_df.dropna(subset=['abstract'])
journal_df[journal_df['abstract'].isna()] 

,authors,title,abstract,journal_name


In [27]:
def clean_content(text):
    # Remove non-alphanumeric characters and convert to lowercase
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text.lower())
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)
    
    return text

# Assuming you have a DataFrame called 'df' with a 'content' column
journal_df['cleaned_abstract'] = journal_df['abstract'].apply(clean_content)
journal_df.head()

,authors,title,abstract,journal_name,cleaned_abstract
0,Yu-Xiao Wang; Yue Xin; Jun-Yi Yin; Xiao-Jun Hu...,Revealing the architecture and solution proper...,Macrolepiota albuminosa (Berk.) Pegler is abun...,Food Chemistry,macrolepiota albuminosa berk pegler is abun...
1,Yu-Xiao Wang; Yue Xin; Xiao-Jun Huang; Jun-Yi ...,A branched galactoglucan with flexible chains ...,A homogeneous galactoglucan was purified from ...,Food Chemistry,a homogeneous galactoglucan was purified from ...
2,Yu-Xiao Wang; Ting Zhang; Jun-Yi Yin; Xiao-Jun...,Structural characterization and rheological pr...,A homogeneous beta-glucan (JHMP-70) was obtain...,Food Hydrocolloids,a homogeneous beta glucan jhmp 70 was obtain...
3,Yu-Xin Gu; Tian-Ci Yan; Zi-Xuan Yue; Min-Hui L...,Dispersive micro-solid-phase extraction of aca...,A novel dispersive micro-solid-phase extractio...,Food Analytical Methods,a novel dispersive micro solid phase extractio...
4,Yu-Xue Xu; Ze-Dong Jiang; Xi-Ping Du; Ming-Jin...,The identification of biotransformation pathwa...,The yeast Saccharomyces cerevisiae is effectiv...,Food Chemistry,the yeast saccharomyces cerevisiae is effectiv...
